In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
import seaborn as sns

#print('Running on PyMC3 v{}'.format(pm.__version__))

# If you get an error like: non-constant-expression cannot be narrowed 
# then comment these lines out
# If you know how to actually fix it, please let me know!
#import theano
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"


# Loading a Saved Trace
We can save traces off to disk and reload them later, and do all the post-sampling analysis and prediction on the re-loaded trace.

This notebook shows how to do that.

First, we will load the training dataset and a holdout dataset

In [ ]:
observed_usage = pd.read_csv('data/training.csv').iloc[:,1:]
holdout = pd.read_csv('data/holdout.csv').iloc[:,1:4]


Then we need to pull in a little bit of code from the original notebook. We need the data generation function of course

In [ ]:
def generate_data(Q, PI, th0, G, ALPHA, num_samples, renewal_period, force_no_churn, initial_state=None):

    states = []
    samples = []
    events = []

    # construct Theta vector from parameters
    THETA = np.zeros(Q.shape[0])
    THETA[0] = th0
    for i in range(1,Q.shape[0]):
        THETA[i] = THETA[i-1] + np.exp(G[i-1])
    
    for i in range(0,ALPHA.shape[0]):
        if initial_state is None:
            state = np.random.choice(Q.shape[0], 1, p=Q)[0]
        else:
            state = np.random.choice(Q.shape[0],1,p=PI[initial_state[i]])[0]     

        churned = False
        for j in range(0,num_samples):  

            # if the force_no_churn flag is set, and churn condition met
            # the choose a new state
            if churned == False and force_no_churn == True and state == 0 and (j+1) % renewal_period == 0:
                while state == 0:
                    state = np.random.choice(Q.shape[0],1,p=PI[state])[0]

            # save current state
            states = np.append(states, state)

            # usage calculation in current state
            if churned == False:
                lam = ALPHA[i] * THETA[state]
                sample = np.random.poisson(lam=lam)
            else:
                sample = 0

            # churn event in current state
            if churned == False:
                if state == 0 and (j+1) % renewal_period == 0:
                    churned = True
                    event = 1
                else:
                    event = 0
            else:
                event = 0

            samples = np.append(samples, sample)
            events = np.append(events, event)

            # choose next state
            if churned == False:
                state = np.random.choice(Q.shape[0],1,p=PI[state])[0]
            else:
                state = 0

    states = np.reshape(states, (ALPHA.shape[0], num_samples))
    events = np.reshape(events, (ALPHA.shape[0], num_samples))
    samples = np.reshape(samples, (ALPHA.shape[0], num_samples))
    return [states, events, samples]

We also need our custom distribution classes, because we'll also need a model.

In [ ]:
from pymc3.distributions.dist_math import bound

class CommitmentProcess(pm.Categorical):
    """
    
    """

    def __init__(self, PI=None, Q=None, renewal_mask=None, num_states=3, 
                 *args, **kwargs):
        super(pm.Categorical, self).__init__(*args, **kwargs)
        self.PI = PI
        self.Q = Q
        self.renewal_mask = renewal_mask
        self.k = num_states
        self.mode = tt.cast(0,dtype='int64')
    
    def logp(self, x):
        
        log_p = 0.
        for i in range(0, self.shape[0]):
            p_it = self.PI[x[i,][:-1]]
            x_t = x[i,][1:]
            x_0 = tt.stack([x[i,][0]])
            mask = self.renewal_mask
        
            log_p_i = pm.Categorical.dist(self.Q).logp(x_0) + tt.sum(pm.Categorical.dist(p_it).logp(x_t))
        
            # Restrction: if not churned, cannot be in state 0 at a renewal period
            log_p = log_p + bound(log_p_i, tt.dot(mask, x_t)>0)

        return log_p


In [ ]:
class UsageProcess(pm.Discrete):
    """
    
    """

    def __init__(self, alpha=None, th0=None, G=None, states=None, num_states=3,
                 *args, **kwargs):
        super(UsageProcess, self).__init__(*args, **kwargs)
        self.alpha = alpha
        self.th0 = th0
        self.G = G
        self.states = states
        self.num_states = num_states
        self.mean = 0.

        
    def logp(self, x):
        states = self.states

        # build theta vector from the components
        theta = tt.concatenate([tt.stack([self.th0]), tt.exp(self.G)])
        zero = tt.zeros_like(theta)
        
        for i in range(1, num_states):
            theta = theta + tt.concatenate([zero[-i:], theta[0:self.num_states-i]])
        
        # build lambda matrix: theta is row vector, alpha is column
        # labmda is the outer-product of the two
        Lambda = tt.outer(self.alpha,theta)
        
        log_p = 0.
        for i in range(0, self.shape[0]):
            lam_it = Lambda[i][states[i,]]
            y_it = x[i]
            log_p_i = tt.sum(pm.Poisson.dist(mu=lam_it).logp(y_it))
            
            log_p = log_p + log_p_i

        return log_p


## Model
We do have to define a model with the same variables, but we don't actually need to run sampling. 

In [ ]:
post_burn_in=1000

renewal_period = 3
num_states = 2

num_custs = observed_usage.shape[0]
obs_len = observed_usage.shape[1]
renewal_mask = np.where(((np.arange(1,obs_len)+1) % renewal_period)!=0,0,1)

states_test_val = np.ones((num_custs, obs_len))


with pm.Model() as model:
    Q = pm.Dirichlet('Q', a=np.ones((num_states)) + 1., shape=(num_states))    
    PI = pm.Dirichlet('PI', a=np.ones((num_states,num_states)) + 1., shape=(num_states,num_states))    
    r = pm.Gamma('r', alpha=0.01, beta=0.01)
    A = pm.Gamma('A',alpha=r, beta=r, shape=((num_custs)))    
    th0 = pm.Uniform('th0',lower=0.0,upper=10.0)
    G = pm.Normal('G',mu=np.zeros(num_states-1), sd=np.ones(num_states-1)*10000., shape=(num_states-1))

    states = CommitmentProcess('states', PI=PI, Q=Q, renewal_mask = renewal_mask, num_states=num_states, shape=(num_custs,obs_len), testval=states_test_val)
    usage = UsageProcess('usage', alpha=A, th0=th0, G=G, states=states, num_states=num_states, shape=(num_custs), observed=observed_usage)
    


## Load the Trace
It's one line! And after it's loaded we can do analysis and prediction

In [ ]:
trace = pm.backends.ndarray.load_trace('./traces/ta', model=model)

## Model Convergence
From the paper
>We burn-in the first 95,000 iterations and use the following 5,000 iterations to draw from the posterior marginal distributions. Convergence of the parameters was diagnosed visually, looking at the historical draws of all parameters, and also using the Gelman-Rubin statistic R, which compares the ratio of the pooled chain variance with the within chain variance, for a model estimated using multiple and disperse starting values.

We can use the trace plot to visually diagnose convergence

In [ ]:
#_ = pm.traceplot(trace[-post_burn_in:])

#from pymc3.diagnostics import gelman_rubin
#gelman_rubin(trace, include_transformed=True)

#model_waic = pm.waic(trace, model=model, progressbar=True)

<a id="prediction"></a>
## Prediction
Now that we've estimated the parameters of the model, we can use them to predict future customer behavior.

### Point Estimate
One source of estimates is to take the average of the trace values (ignoring some initial section of the trace, as "burn-in").

In [ ]:
Q_avg = np.average(trace['Q'][-post_burn_in:],axis=0)
PI_avg = np.average(trace['PI'][-post_burn_in:],axis=0)
G_avg = np.average(trace['G'][-post_burn_in:],axis=0)
th0_avg = np.average(trace['th0'][-post_burn_in:],axis=0)
A_avg = np.average(trace['A'][-post_burn_in:],axis=0)
last_state = np.floor(np.average(trace['states'][-post_burn_in:],axis=0)[:,-1] + 0.5).astype(int)

### State Estimation
Since the last time period in the data represents a renewal period, we can predict who will churn based on the estimate of the last state

In [ ]:
A_avg

We can also plot a distribution for that estimated state to get an idea of the uncertainty

In [ ]:
_ = sns.distplot(trace['states'][-post_burn_in:,0,-1])

Now we can take these paramater estimates and give them to the data generator to predict future states and usage. This is where the `initial_state` param comes in: we can use an estimate of the last state each customer was in.

In [ ]:
prediction_periods = 3
prediction = generate_data(Q_avg, PI_avg, th0_avg, G_avg, A_avg, prediction_periods, renewal_period, False, last_state)
#np.reshape(prediction, (3, A_avg.shape[0], prediction_periods))

We can compare the predictions with held out data

In [ ]:
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(holdout.values, prediction[2]))

### Sample from the Trace
Instead of taking the average of the trace, we can iterate over the trace data and generate a prediction for each of the parameter values. This will give us a result that shows the uncertainty of the prediction.

In [ ]:
states = []
events = []
usage = []

prediction_periods = 3
for i in range(post_burn_in):
    Q_sample = trace['Q'][i]
    PI_sample = trace['PI'][i]
    G_sample = trace['G'][i]
    th0_sample = trace['th0'][i]
    A_sample = trace['A'][i]
    last_observed_state = trace['states'][i][:,-1]
    st, ev, us = generate_data(Q_sample, PI_sample, th0_sample, G_sample, A_sample, prediction_periods, renewal_period, False, last_observed_state )
    states.append(st)
    events.append(ev)
    usage.append(us)
    
states = np.reshape(states, (post_burn_in, A_sample.shape[0], prediction_periods))
events = np.reshape(events, (post_burn_in, A_sample.shape[0], prediction_periods))
usage = np.reshape(usage, (post_burn_in, A_sample.shape[0], prediction_periods))

This gives us a lot of possible outcomes. We can do things like plot the distribution of the predicted usage for one customer in a certain period

In [ ]:
cust_id=7
period=0
_ = sns.distplot(usage[:,cust_id,period])
_ = plt.axvline(x=holdout.iloc[cust_id,period], color='r')